In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Once the initial setup is complete, go to <a href = "https://www.makesense.ai/">MakeSense AI </a> and start the image labeling process. yolov5 requires annotated labels in a certain format.

This can be done in AWS Sagemake Ground Truth as well. However, Amazon Sagemaker Ground Truth will generate an augmented manifest file, which will need to be converted to yolov5 format. 

In [ ]:
%cd yolov5/

In [ ]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y

Go to yolov5/data and download the coco128.yaml file and make the following changes

- Set the path to your trianing data
- Set the path to your validation data
- Set the number of classes. In our case we are predicting hard hats and hence number of classes is 1
- Set the name of the label. In our case, it's only 1 and hence remove all other labels

Once you make these changes save the file as custom_data.yaml and upload it to yolov5/data folder


Use the train.py script to start training your model 

- img 416 indicates the size of the image
- batch 1 indicates 1 file is sent at a time for training. You can modify this based on the resources you have.
- epochs 30 indicates that we will train the model for 30 passes. You can increase this number for better accuracy
- custom_data.yaml has paths for our train and test data as well as the label details
- We start with yolov5 pretrained weights. This is an example of transfer learning

The model will train for the number of set epochs and at the end of the training will provide the trained weights that we will use

In [ ]:
!python train.py --img 416 --batch 1 --epochs 30 --data custom_data.yaml --weights yolov5s.pt --cache

Once training is complete, you can go to yolov5/runs/detect/exp to check for the training results 

runs/train/exp5/weights/last.pt

In [ ]:
!python detect.py  --weights runs/train/exp5/weights/last.pt --img 640 --conf 0.25 --source ../Manufacturing\ Safety\ Employee\ Video.mp4

Use the detect.py script to detect hardhats in a video or an image

- weights indicate our best trained weights after "n" epochs
- img 640 will set the video frame size
- conf 0.25 indicates show all detections with a confidence score of 25% and above
- source is the source of the data we need to run the detection on

Once the detection on the frames is complete, upload the video to the S3 bucket and then download it to your machine

In [ ]:
! aws s3 cp runs/detect/exp/Manufacturing\ Safety\ Employee\ Video.mp4 s3://ground-truth-labeling-test-v1/